In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('preprocessed_reviews.csv')

# Get the number of reviews for each product
product_review_count = df['app_name'].value_counts()


print(product_review_count)


app_name
Dota 2                            73541
Rocket League                     54227
The Witcher 3: Wild Hunt          31853
DOOM                              20682
Counter-Strike                    12353
Call of Duty: Modern Warfare 3     5197
NBA 2K16                           2534
The Elder Scrolls V: Skyrim        2260
Football Manager 2016              1742
Name: count, dtype: int64


In [3]:
df.head()

,app_id,app_name,review_text,review_score,review_votes,tokens,sentiment,cleaned_review_text
0,10,Counter-Strike,ruined my life,1,0,"['ruined', 'life']",-0.4767,ruined life
1,10,Counter-Strike,this will be more of a my experience with this...,1,1,"['experience', 'game', 'type', 'review', 'sayi...",0.9961,experience game type review saying things like...
2,10,Counter-Strike,this game saved my virginity,1,0,"['game', 'saved', 'virginity']",0.4215,game saved virginity
3,10,Counter-Strike,do you like original games do you like games...,1,0,"['like', 'original', 'games', 'like', 'games',...",0.8817,like original games like games dont lag like g...
4,10,Counter-Strike,easy to learn hard to master,1,1,"['easy', 'learn', 'hard', 'master']",0.3612,easy learn hard master


In [3]:
""" import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Define a function to preprocess text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

# Apply the function to the review_text column
df['cleaned_review_text'] = df['review_text'].apply(lambda x: preprocess_text(x) if isinstance(x, str) else x)

print(df[['review_text', 'cleaned_review_text']].head()) """

" import string\nfrom nltk.corpus import stopwords\nfrom nltk.tokenize import word_tokenize\nimport nltk\n\n# Download stopwords if not already downloaded\nnltk.download('stopwords')\nnltk.download('punkt')\n\n# Define a function to preprocess text\ndef preprocess_text(text):\n    # Convert to lowercase\n    text = text.lower()\n    # Remove punctuation\n    text = text.translate(str.maketrans('', '', string.punctuation))\n    # Tokenize text\n    tokens = word_tokenize(text)\n    # Remove stopwords\n    tokens = [word for word in tokens if word not in stopwords.words('english')]\n    return ' '.join(tokens)\n\n# Apply the function to the review_text column\ndf['cleaned_review_text'] = df['review_text'].apply(lambda x: preprocess_text(x) if isinstance(x, str) else x)\n\nprint(df[['review_text', 'cleaned_review_text']].head()) "

In [4]:
# Define a function to detect sarcasm
def detect_sarcasm(row):
    # Check for rating-text mismatch
    if row['review_score'] == 1 and row['sentiment'] < 0:
        return 1
    else:
        return 0

# Apply the function to the DataFrame
df['sarcasm'] = df.apply(detect_sarcasm, axis=1)

print(df[['cleaned_review_text', 'review_score', 'sentiment', 'sarcasm']].head())



                                 cleaned_review_text  review_score  sentiment  \
0                                        ruined life             1    -0.4767   
1  experience game type review saying things like...             1     0.9961   
2                               game saved virginity             1     0.4215   
3  like original games like games dont lag like g...             1     0.8817   
4                             easy learn hard master             1     0.3612   

   sarcasm  
0        1  
1        0  
2        0  
3        0  
4        0  


In [5]:
print(df['sarcasm'].value_counts())


sarcasm
0    182283
1     22106
Name: count, dtype: int64


In [6]:
print(df['cleaned_review_text'].head(10))


0                                          ruined life
1    experience game type review saying things like...
2                                 game saved virginity
3    like original games like games dont lag like g...
4                               easy learn hard master
5                                      r revolver play
6                        still better call duty ghosts
7    cant buy skins cases keys stickers gaben cant ...
8    counterstrike ok years unlimited fun friends f...
9    every server spanish french fluently swear lan...
Name: cleaned_review_text, dtype: object


In [7]:
print(df['review_votes'].value_counts())

review_votes
0    173422
1     30967
Name: count, dtype: int64


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import scipy.sparse

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
# Fit and transform the cleaned_review_text column
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_review_text'].dropna())

# Convert the TF-IDF matrix to a DataFrame (using sparse matrix format)
tfidf_df = pd.DataFrame.sparse.from_spmatrix(tfidf_matrix, columns=tfidf_vectorizer.get_feature_names_out())

print(tfidf_df.head())

   aa  aaa  aaaa  aaaaa  aaaaaaaaaa  aaaaaaaaaaa  \
0   0    0     0      0           0            0   
1   0    0     0      0           0            0   
2   0    0     0      0           0            0   
3   0    0     0      0           0            0   
4   0    0     0      0           0            0   

   aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa  \
0                                 0   
1                                 0   
2                                 0   
3                                 0   
4                                 0   

   aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa  \
0                                                  0                
1                                                  0                
2                                                  0                
3                                                  0                
4                                                  0                

   aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

In [5]:
nonzero_count = tfidf_matrix.nnz
total_elements = tfidf_matrix.shape[0] * tfidf_matrix.shape[1]
print("Non-zero entries:", nonzero_count)
print("Total elements:", total_elements)


Non-zero entries: 3714730
Total elements: 18501935652


In [6]:
# Print the nonzero tokens for the first review
nonzero_tokens = tfidf_df.iloc[0][tfidf_df.iloc[0] != 0]
print(nonzero_tokens)


life      0.570045
ruined    0.821614
Name: 0, dtype: Sparse[float64, 0]


In [7]:
nonzero_per_row = tfidf_matrix.getnnz(axis=1)
print("Nonzero counts for each row:", nonzero_per_row)

df.head()

Nonzero counts for each row: [  2 140   3 ...   3 124  22]


,app_id,app_name,review_text,review_score,review_votes,tokens,sentiment,cleaned_review_text
0,10,Counter-Strike,ruined my life,1,0,"['ruined', 'life']",-0.4767,ruined life
1,10,Counter-Strike,this will be more of a my experience with this...,1,1,"['experience', 'game', 'type', 'review', 'sayi...",0.9961,experience game type review saying things like...
2,10,Counter-Strike,this game saved my virginity,1,0,"['game', 'saved', 'virginity']",0.4215,game saved virginity
3,10,Counter-Strike,do you like original games do you like games...,1,0,"['like', 'original', 'games', 'like', 'games',...",0.8817,like original games like games dont lag like g...
4,10,Counter-Strike,easy to learn hard to master,1,1,"['easy', 'learn', 'hard', 'master']",0.3612,easy learn hard master


In [3]:
# --- Data Preparation ---
# Filter the DataFrame to include only rows with a cleaned review text.
# This ensures that the TF-IDF features align with your target variable.
df_clean = df.dropna(subset=['cleaned_review_text'])
df_clean.drop(columns=['review_text', 'review_votes', 'sentiment'], inplace=True)

# IMPORTANT:
# The tfidf_matrix should have been computed on df['cleaned_review_text'].dropna()
# so its rows correspond to df_clean.
# If that’s not the case, re-run the vectorization on df_clean, e.g.:
# tfidf_matrix = tfidf_vectorizer.fit_transform(df_clean['cleaned_review_text'])

# Define features and target
X = tfidf_matrix          # TF-IDF features (sparse matrix)
y = df_clean['review_score']  # Target variable: 1 (positive/recommend) or -1 (negative/not recommend)

# --- Train-Test Split ---
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- Oversampling using SMOTE ---
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

C:\Users\User\AppData\Local\Temp\ipykernel_16372\2618862361.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(columns=['review_text', 'review_votes', 'sentiment'], inplace=True)


In [9]:
# --- Model Training ---
from sklearn.linear_model import LogisticRegression

# Initialize and train the Logistic Regression classifier
lr = LogisticRegression(max_iter=1000, class_weight='balanced')
lr.fit(X_train_resampled, y_train_resampled)

# --- Model Evaluation ---
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predict on the test set
y_pred = lr.predict(X_test)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.8558147737579361

Classification Report:
              precision    recall  f1-score   support

          -1       0.41      0.84      0.55      4161
           1       0.98      0.86      0.91     35531

    accuracy                           0.86     39692
   macro avg       0.69      0.85      0.73     39692
weighted avg       0.92      0.86      0.88     39692


Confusion Matrix:
[[ 3506   655]
 [ 5068 30463]]


In [10]:
# Check the shape of tfidf_matrix
print("Shape of tfidf_matrix:", tfidf_matrix.shape)

# Check the number of rows in df_clean
print("Number of rows in df_clean:", df_clean.shape[0])

# Verify if they match
if tfidf_matrix.shape[0] == df_clean.shape[0]:
    print("tfidf_matrix was computed on df_clean['cleaned_review_text']")
else:
    print("tfidf_matrix was NOT computed on df_clean['cleaned_review_text']")

Shape of tfidf_matrix: (198459, 93228)
Number of rows in df_clean: 198459
tfidf_matrix was computed on df_clean['cleaned_review_text']


In [ ]:
from sklearn.svm import SVC

# Initialize and train the SVM classifier
svm = SVC(kernel='linear', class_weight='balanced', random_state=42)
svm.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred_svm = svm.predict(X_test)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_svm))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_svm))

In [22]:
import joblib

# Export the SVM model to a file
joblib.dump(svm, 'svm_model.pkl')

['svm_model.pkl']

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred_rf = rf.predict(X_test)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

Accuracy: 0.9312707850448453

Classification Report:
              precision    recall  f1-score   support

          -1       0.67      0.68      0.67      4161
           1       0.96      0.96      0.96     35531

    accuracy                           0.93     39692
   macro avg       0.82      0.82      0.82     39692
weighted avg       0.93      0.93      0.93     39692


Confusion Matrix:
[[ 2810  1351]
 [ 1377 34154]]


In [14]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize and train the Gradient Boosting classifier
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred_gb = gb.predict(X_test)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred_gb))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_gb))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_gb))

Accuracy: 0.7565504383754913

Classification Report:
              precision    recall  f1-score   support

          -1       0.27      0.77      0.40      4161
           1       0.97      0.75      0.85     35531

    accuracy                           0.76     39692
   macro avg       0.62      0.76      0.62     39692
weighted avg       0.89      0.76      0.80     39692


Confusion Matrix:
[[ 3220   941]
 [ 8722 26809]]


In [15]:
import joblib

# Export the Random Forest model to a file
joblib.dump(rf, 'random_forest_model.pkl')

# Export the Gradient Boosting model to a file
joblib.dump(gb, 'gradient_boosting_model.pkl')

['gradient_boosting_model.pkl']

In [7]:
df_clean.head()

,app_id,app_name,review_score,tokens,sentiment,cleaned_review_text,sarcasm
1,10,Counter-Strike,1,"['experience', 'game', 'type', 'review', 'sayi...",0.9961,experience game type review saying things like...,0
2,10,Counter-Strike,1,"['game', 'saved', 'virginity']",0.4215,game saved virginity,0
3,10,Counter-Strike,1,"['like', 'original', 'games', 'like', 'games',...",0.8817,like original games like games dont lag like g...,0
4,10,Counter-Strike,1,"['easy', 'learn', 'hard', 'master']",0.3612,easy learn hard master,0
5,10,Counter-Strike,1,"['r', 'revolver', 'play']",0.0516,r revolver play,0


In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Load BERT tokenizer and model
#tokenizer = BertTokenizer.from_pretrained("./saved_model")
#model = BertForSequenceClassification.from_pretrained("./saved_model")

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["cleaned_review_text"], padding="max_length", truncation=True, max_length=128)

df_clean['review_score'] = df_clean['review_score'].apply(lambda x: 1 if x == 1 else 0)

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\User\AppData\Local\Temp\ipykernel_16372\2893596847.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['review_score'] = df_clean['review_score'].apply(lambda x: 1 if x == 1 else 0)


In [5]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available

# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df_clean)
dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.remove_columns(["tokens"])
dataset = dataset.rename_column("review_score", "labels")
dataset.set_format("torch")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Split dataset
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

True


Map: 100%|██████████| 198459/198459 [01:18<00:00, 2523.54 examples/s]


In [14]:
print(dataset)

Dataset({
    features: ['app_id', 'app_name', 'labels', 'cleaned_review_text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 198459
})


In [10]:
from sklearn.metrics import accuracy_score, classification_report
def compute_metrics(eval_pred):
    logits, labels = eval_pred  # Unpack predictions and actual labels
    predictions = np.argmax(logits, axis=-1)  # Convert logits to class labels (0 or 1)
    return {"accuracy": accuracy_score(labels, predictions)}  # Calculate accuracy

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True if torch.cuda.is_available() else False
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train model
trainer.train() 

eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

C:\Users\User\AppData\Local\Temp\ipykernel_16372\464535618.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.288700,0.340627,0.894966
2,0.339300,0.334525,0.894966
3,0.263500,0.287105,0.912174


Evaluation results: {'eval_loss': 0.28710535168647766, 'eval_accuracy': 0.912173737780913, 'eval_runtime': 49.5607, 'eval_samples_per_second': 800.877, 'eval_steps_per_second': 50.06, 'epoch': 3.0}


In [31]:
from sklearn.metrics import accuracy_score
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Compute accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Get predicted class (0 or 1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Ensure Trainer uses compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics  # Include this
)

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")



c:\Python312\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\User\AppData\Local\Temp\ipykernel_2012\3187048647.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Python312\Lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Evaluation results: {'eval_loss': 0.33828869462013245, 'eval_model_preparation_time': 0.002, 'eval_accuracy': 0.9122533454298026, 'eval_runtime': 58.3698, 'eval_samples_per_second': 604.286, 'eval_steps_per_second': 75.536}


In [32]:
print(eval_results['eval_loss'])

0.33828869462013245


In [33]:
# Compute accuracy
predictions = np.argmax(eval_results["eval_loss"], axis=-1)  # Convert logits to class labels


accuracy = accuracy_score(test_dataset["labels"], predictions)

print(f"Evaluation results: {eval_results}")
print(f"Model Accuracy: {accuracy:.4f}")

InvalidParameterError: The 'y_pred' parameter of accuracy_score must be an array-like or a sparse matrix. Got 0 instead.

In [11]:
# Save model and tokenizer
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")
print("Model saved successfully.")

Model saved successfully.


In [1]:
import torch
import accelerate
import transformers
print(torch.__version__)
print(accelerate.__version__)
print(transformers.__version__)

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.4.0+cpu
0.34.2
4.49.0


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    
    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return prediction

# Apply the trained model to predict sentiment for all reviews
df_clean['predicted_sentiment'] = df_clean['cleaned_review_text'].apply(predict_sentiment)

df

C:\Users\User\AppData\Local\Temp\ipykernel_16372\4036765384.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['predicted_sentiment'] = df_clean['cleaned_review_text'].apply(predict_sentiment)


,app_id,app_name,review_score,tokens,cleaned_review_text,predicted_sentiment
0,10,Counter-Strike,1,"['ruined', 'life']",ruined life,1
1,10,Counter-Strike,1,"['experience', 'game', 'type', 'review', 'sayi...",experience game type review saying things like...,1
2,10,Counter-Strike,1,"['game', 'saved', 'virginity']",game saved virginity,1
3,10,Counter-Strike,1,"['like', 'original', 'games', 'like', 'games',...",like original games like games dont lag like g...,1
4,10,Counter-Strike,1,"['easy', 'learn', 'hard', 'master']",easy learn hard master,1


In [17]:
df_clean[df_clean['predicted_sentiment'] != df_clean['review_score']].head()

df_clean.to_csv('predicted_sentiment.csv', index=False)